In [1]:
import pandas as pd
import numpy as np
from tabula import read_pdf

In [2]:
def mergeTables(dfs, start):
    dfMerged = pd.DataFrame()

    for i, df in enumerate(dfs):
        year = start - i
        df["year"] = year
        dfMerged = pd.concat([dfMerged, df], ignore_index = True)

    # arrange the order of columns
    cols = list(dfMerged.columns)
    yearIdx = cols.index("year")
    cols.pop(yearIdx)
    cols.append("year")
    dfMerged = dfMerged[cols]

    return dfMerged

def mergeColumns(df, targets, newColName, mergeFunc):
    df["temp"] = ""
    for i in range(len(targets)):
        mergeFunc(df, targets, i)
    
    df.drop(columns = targets, inplace = True)
    df.insert(0, newColName, df["temp"])
    df.drop(columns = ["temp"], inplace = True)
    return df

## Boston

In [3]:
boston_df = pd.read_csv("data/Boston/boston_police_2013_to_2018.csv", index_col = [0])

## Somerville

In [4]:
def somervilleMerge(df, targets, i):
    df["temp"] += df[targets[i]]

In [5]:
somerville_2014_df = pd.read_csv("data/Somerville/City_of_Somerville_Weekly_Payroll_Gross_Wages_Over_50K_2014.csv")
somerville_2015_df = pd.read_csv("data/Somerville/City_of_Somerville_Weekly_Payroll_Gross_Wages_Over_50K_2015.csv")
somerville_2016_df = pd.read_csv("data/Somerville/City_of_Somerville_Weekly_Payroll_Gross_Wages_Over_50K_2016.csv")
somerville_2017_df = pd.read_csv("data/Somerville/City_of_Somerville_Weekly_Payroll_Gross_Wages_Over_50K_2017.csv")
somerville_2018_df = pd.read_csv("data/Somerville/City_of_Somerville_Weekly_Payroll_Gross_Wages_Over_50K_2018.csv")
somerville_dfs = [somerville_2018_df, somerville_2017_df, somerville_2016_df, somerville_2015_df, somerville_2014_df]

In [6]:
# merge tables
somerville_2013_to_2018_df = mergeTables(somerville_dfs, 2018)

targets = ["Name", "Employee Name"]
# fill NaN values with "" and merge targets
somerville_2013_to_2018_df[targets] = somerville_2013_to_2018_df[targets].fillna("")
somerville_2013_to_2018_df = mergeColumns(somerville_2013_to_2018_df, targets, "Name", somervilleMerge)
# change all names to uppercase ones
somerville_2013_to_2018_df["Name"] = somerville_2013_to_2018_df["Name"].str.upper()

## Springfield

In [7]:
def springfieldMerge(df, targets, i):
    df["temp"] += (df[targets[i]] + ", " if i != len(targets) - 1 else df[targets[i]])

In [8]:
springfield_2013_df = pd.read_excel("data/Springfield/Springfield 2013.xls", skiprows = [0])
springfield_2014_df = pd.read_excel("data/Springfield/Springfield 2014.xls", skiprows = [0])
springfield_2015_df = pd.read_excel("data/Springfield/Springfield 2015.xlsx", skiprows = [0])
springfield_2016_df = pd.read_excel("data/Springfield/Springfield 2016.xlsx", skiprows = [0])
springfield_2017_df = pd.read_excel("data/Springfield/Springfield 2017.xlsx", skiprows = [0])
springfield_2018_df = pd.read_excel("data/Springfield/Springfield 2018.xlsx", skiprows = [0])
springfield_dfs = [springfield_2018_df, springfield_2017_df, springfield_2016_df, springfield_2015_df, springfield_2014_df, springfield_2013_df]

In [9]:
# merge tables
springfield_2013_to_2018_df = mergeTables(springfield_dfs, 2018)

targets = ["First Name", "Last Name"]
# fill NaN values with "" and merge targets
springfield_2013_to_2018_df[targets] = springfield_2013_to_2018_df[targets].fillna("")
springfield_2013_to_2018_df = mergeColumns(springfield_2013_to_2018_df, targets, "Name", springfieldMerge)
# change all names to uppercase ones
springfield_2013_to_2018_df["Name"] = springfield_2013_to_2018_df["Name"].str.upper()

In [10]:
somerville_2013_to_2018_df.to_csv("data/Somerville/somerville_police_2014_to_2018.csv")
springfield_2013_to_2018_df.to_csv("data/Springfield/springfield_police_2013_to_2018.csv")